In [ ]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

#Function to print the information about data set

In [ ]:
def data_info(train_data1):
  print(train_data1.isnull().sum())
  print(train_data1.info())
  print(train_data1.shape)
  # print(train_data1.Label.value_counts())

#Function to preprocess the text data

In [ ]:
def text_preprocessing(data):
  # remove punctuations
  text = data.translate(str.maketrans('','',string.punctuation))
  # lower case text
  text = text.lower()
  # remove number
  text = re.sub(r'\d+', '', text)
  # remove hyperlink
  text = re.sub(r'http\S+','',text)
  # remove non english text
  pattern = r'[^a-zA-Z ]'
  text = re.sub(pattern,'', text)
  # remove extra space
  text = re.sub('\s+',' ',text)
  # tokanise text
  tokens = nltk.word_tokenize(text)
  # remove stop words
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]
  # lemmatize word
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  # join the tokens back to string
  processed_text = ' '.join(tokens)
  return processed_text

In [ ]:
train_data = pd.read_csv("/content/train_data.csv", sep=',')
df = train_data.drop(labels='PID',axis=1)
label_train = df.Label
# test_data = pd.read_csv("/content/test_data.csv",sep=',',encoding='latin1')
# df2 = test_data.drop(labels='Pid',axis=1)
# df['Text'] = df['Text'].apply(text_preprocessing)
# df2['Text'] = df2['Text'].apply(text_preprocessing)
valid_data = pd.read_csv("/content/dev_data.csv")
df3 = valid_data.drop(labels ="Pid", axis =1)
label_valid = df3.Labels
# df3['Text'] = df3['Text'].apply(text_preprocessing)

# df2
# # Filter rows with NaN values
# nan_rows = df2[df2.isnull().any(axis=1)]

# # Print the rows with NaN values
# print(nan_rows)
# data_info(df2)
# data_info(df)

In [ ]:
!pip install transformers

# Downloading pretrained model

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import TensorDataset, DataLoader
from transformers import XLNetTokenizer, XLNetModel

def get_embeddings(dataframe,max_length=80, batch_size=16):
    # Load PubMedBERT tokenizer and model
    # tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
    # model = AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")

    # to load mental bert model
    # tokenizer = AutoTokenizer.from_pretrained("mental/mental-bert-base-uncased")
    # model = AutoModel.from_pretrained("mental/mental-bert-base-uncased")

    #  Load Clinical BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    model = BertModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

    # move model to Gpu device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)


    # Tokenize and pad the input text
    tokenized = dataframe["Text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=max_length)))
    tokenized = [torch.tensor(tokens) for tokens in tokenized]
    padded = torch.nn.utils.rnn.pad_sequence(tokenized, batch_first=True).to(device)


    # Create attention masks
    attention_masks = padded.gt(0).to(device)

    # Create TensorDataset
    dataset = TensorDataset(padded, attention_masks)

    # Create DataLoader for batching
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Retrieve embeddings using Clinical BERT model
    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            batch_embeddings = outputs.last_hidden_state[:, 0, :]
            embeddings.append(batch_embeddings)

    # Concatenate embeddings from all batches
    embeddings = torch.cat(embeddings, dim=0).cpu()
    # # Reshape embeddings to 2D tensor
    # embeddings = embeddings.view(embeddings.shape[0], -1)


    # # Create a new dataframe to store the embeddings
    # embedding_df = pd.DataFrame(embeddings.numpy())

    return embeddings

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install SentencePiece

#building model for bert

In [ ]:
# Example usage
# Assuming you have a preprocessed dataframe called 'preprocessed_df'
import pickle
# path = 'C:/Users/shankar/Desktop/pickle/embeddings.pkl'
# embeddings_dftrain = get_embeddings(df, max_length=80, batch_size=16)
# embeddings_dftest = get_embeddings(df2, max_length=80, batch_size=16)
embedding_valid = get_embeddings(df3,max_length=80, batch_size=16)


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# dftrain = embeddings_dftrain.numpy()
# dftest = embeddings_dftest.numpy()
dfvalid = embedding_valid.numpy()
dfvalid.shape

(3245, 768)

code to store embeddings in pickle file

In [ ]:
# clinical_bert = embeddings_dftrain.numpy()
pickle_out = open("clinicalbert_valid.pickle","wb")
pickle.dump(dfvalid, pickle_out)
pickle_out.close()


Retriving pickle values

In [ ]:
import pickle
# retrive train data
pickle_in = open("clinical_berttrain.pickle", "rb")
clinical_train = pickle.load(pickle_in)
# retrive valid data
pickle_in = open("clinicalbert_valid.pickle", "rb")
clinical_valid = pickle.load(pickle_in)


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y_train = label_encoder.fit_transform(label_train)
Y_valid = label_encoder.fit_transform(label_valid)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define the BiLSTM Classifier model
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiLSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes

        self.embedding = nn.input(input_size, hidden_size)
        self.bilstm = nn.LSTM(hidden_size, hidden_size, num_layers, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x, lengths):
        embedded = self.input(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.to('cpu'), batch_first=True, enforce_sorted=False)
        output, _ = self.bilstm(packed)
        output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        output = self.fc(output[:, -1, :])
        return output

# Convert data to tensors
X_text = torch.tensor(clinical_train, dtype=torch.float64).to(device)
X_label = torch.tensor(Y_train, dtype=torch.long).to(device)
Y_text = torch.tensor(clinical_valid, dtype=torch.float64).to(device)
Y_label = torch.tensor(Y_valid, dtype=torch.long).to(device)
# Create DataLoader for training and testing sets
train_dataset = torch.utils.data.TensorDataset(X_text, X_label)
test_dataset = torch.utils.data.TensorDataset(Y_text, Y_label)
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the BiLSTM Classifier model
input_size = 768  # Dimension of the input embeddings
hidden_size = 120
num_layers = 2
num_classes = 3
model = BiLSTMClassifier(input_size, hidden_size, num_layers, num_classes)
model = model.to(device)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



NameError: ignored

In [ ]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    # model.train()
    train_loss = 0.0
    for inputs, labels in train_dataloader:
        lengths = torch.sum(inputs != 0, dim=1).to(torch.long)
        inputs = inputs.to(device)
        labels = labels.to(device)
        print(inputs,labels)
        # Ensure that the indices are within the valid range
        inputs = torch.clamp(inputs, 0, input_size - 1)


        optimizer.zero_grad()

        outputs = model(inputs, lengths)
        loss = criterion(outputs, labels)
        train_loss += loss.item() * inputs.size(0)

        loss.backward()
        optimizer.step()

    train_loss /= len(train_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}")
    #  Print some intermediate values for debugging
    print("Inputs:", inputs)
    print("Labels:", labels)
    print("Outputs:", outputs)



tensor([[ 0.2639,  0.0337, -0.7855,  ..., -0.1113,  0.1930, -0.0935],
        [ 0.1384,  0.1091, -0.4525,  ...,  0.0367,  0.1303, -0.5447],
        [ 0.1024, -0.0171, -0.5972,  ...,  0.1391, -0.0145, -0.4238],
        ...,
        [-0.2187,  0.1587, -0.6291,  ...,  0.0814,  0.1888, -0.1995],
        [-0.0163,  0.2741, -0.8338,  ..., -0.1921,  0.3136, -0.2579],
        [-0.1544,  0.3012, -0.8363,  ...,  0.1182,  0.0851, -0.3229]],
       device='cuda:0', dtype=torch.float64) tensor([2., 0., 0., 2., 2., 1., 0., 0., 0., 0., 2., 2., 0., 0., 0., 0.],
       device='cuda:0', dtype=torch.float64)


RuntimeError: ignored

In [ ]:
# Evaluation
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_dataloader:
        lengths = torch.sum(inputs != 0, dim=1).to(torch.long)

        # Ensure that the indices are within the valid range
        inputs = torch.clamp(inputs, 0, input_size - 1)
        inputs = inputs.to(device)
        lengths = lengths.to(device)
        labels = labels.to(device)
        outputs = model(inputs, lengths)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs, dim=1)
        print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_dataset)
accuracy = correct / total * 100

print(f"Test Loss: {test_loss:.4f}")
print(f"Accuracy: {accuracy:.2f}%")


In [ ]:
print(predicted.shape)

torch.Size([13])
